In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

**You trace the path each wire takes as it leaves the central port, one wire per line of text.**

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit,
**you need to find the intersection point closest to the central port**. Because the wires are
on a grid, use the Manhattan distance for this measurement. While the wires do technically
cross right at the central port where they both start, this point does not count, nor does
a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right
8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port:
its distance is 3 + 3 = 6.

Here are a few more examples:

- `R75,D30,R83,U83,L12,D49,R71,U7,L72 U62,R66,U55,R34,D71,R55,D58,R83` = distance 159
- `R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51,U98,R91,D20,R16,D67,R40,U7,R15,U6,R7` = distance 135

What is the Manhattan distance from the central port to the closest intersection?

In [1]:
import numpy as np

def diff(instruction):
    dir, n = instruction[0], int(instruction[1:])
    result = np.array([0, 0])
    slot = 0 if dir in "RL" else 1
    sign = -1 if dir in "DL" else 1
    result[slot] = sign * n
    return result

def instructions_to_coordinates(instructions):
    curr = np.array([0, 0])
    points = [curr.copy()]
    for instruction in instructions:
        delta = diff(instruction)
        curr += delta
        points.append(curr.copy())
    return np.array(points)

def coords_to_layer(coords, h, w):
    layer = np.zeros((w, h), dtype=np.int)
    for (x0, y0), (x1, y1) in zip(coords[:-1], coords[1:]):
        (c0, c1), (r0, r1) = sorted((x0, x1)), sorted((y0, y1))
        if c0 == c1:
            layer[r0:r1+1, c0] = 1
        elif r0 == r1:
            layer[r0, c0:c1+1] = 1
        else:
            raise ValueError
    return layer

def text_to_all_coords(text):
    all_coords = []
    for line in text.splitlines():
        instructions = line.strip().split(",")
        coords = instructions_to_coordinates(instructions)
        all_coords.append(coords)
    return all_coords

def all_coords_to_layers(all_coords):
    center = -np.vstack(all_coords).min(axis=0)
    for coords in all_coords:
        coords += center
    h, w = np.vstack(all_coords).max(axis=0) + 1
    layers = [coords_to_layer(coords, h, w) for coords in all_coords]
    return layers

def intersections(layers):
    intersects = np.ones_like(layers[0])
    for layer in layers:
        intersects &= layer
    return np.argwhere(intersects)[:, ::-1]  # turn row/col back to y/x

def intersection_dists(points, starting_point):
    dists = np.sum(np.abs(points - starting_point), axis=1)
    return sorted(dists[dists > 0])

In [2]:
def part1(text):
    all_coords = text_to_all_coords(text)
    starting_point = all_coords[0][0, :]
    layers = all_coords_to_layers(all_coords)
    points = intersections(layers)
    dists = intersection_dists(points, starting_point)
    return dists[0]

In [3]:
assert part1("""R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83""") == 159

assert part1("""R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7""") == 135

In [4]:
%%time
with open("input.txt", "r") as fp:
    text = fp.read()
part1(text)

CPU times: user 1.33 s, sys: 803 ms, total: 2.13 s
Wall time: 2.13 s


308

In [5]:
def part2(text):
    all_coords = text_to_all_coords(text)
    starting_point = all_coords[0][0, :]
    layers = all_coords_to_layers(all_coords)
    points = intersections(layers)
    crossing_points = {tuple(c.ravel()) for c in points if not (c == all_coords[0][0]).all()}
    counters = []

    for cc in all_coords:
        curr = cc[0].copy()
        step = 0
        diffs = np.diff(cc, axis=0)
        crossing_steps = {}
        for i in range(len(diffs)):
            n_steps = np.abs(diffs[i]).sum()
            move = (diffs[i] / n_steps).astype(np.int)
            for j in range(n_steps):
                curr += move
                step += 1
                x, y = curr
                if (x, y) in crossing_points and (x, y) not in crossing_steps:
                    crossing_steps[(x, y)] = step
        counters.append(crossing_steps)

    final_counts = {}
    for k in crossing_points:
        final_counts[k] = 0
        for counter in counters:
            final_counts[k] += counter[k]
    best_combined = min(final_counts.values())
    return best_combined

In [7]:
assert part2("""R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83""") == 610

assert part2("""R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7""") == 410

In [8]:
%%time
with open("input.txt", "r") as fp:
    text = fp.read()
part2(text)

CPU times: user 1.86 s, sys: 709 ms, total: 2.57 s
Wall time: 2.56 s


12934